# Legal Intelligence Rank Monitor


<br>



This jupyter notebook provides the prototype of the rank monitor:

It should:

- Request queries, clicks and their corresponding position on the query list
- Pre-process the data into readable format
- Calculate the NDCG
- Be able to re-run queries, with appropriate access rights and re-calculate the nDCG

<br> 


In [17]:
# Imports

import numpy as np
from sklearn.metrics import f1_score, average_precision_score
import math
import requests
import os

import datetime
from datetimerange import DateTimeRange
from datetime import timedelta  
from dateutil import parser
from urllib import *
import json

from pathlib import Path
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')


from collections import Counter
from tqdm import tqdm_notebook 
tqdm_notebook().pandas()
from tqdm import tqdm_notebook as tqdm

import requests
import urllib

import re
import glob
import time 

import itertools
import pathlib

# Greedy IDE completion 
%config IPCompleter.greedy=True

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; } </style>"))
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

### Javascript for a duplicate line on Ctrl-P
Run this to set a keyboard shortcut do duplicate a line on: Control+P

In [2]:
%%js

Jupyter.keyboard_manager.edit_shortcuts.add_shortcut('Ctrl-p', {
    help : 'Duplicate current line',
    help_index : 'zz',
    handler: function(env) {
        var cm=env.notebook.get_selected_cell().code_mirror;
        
        // get a position of a current cursor in a current cell
        var current_cursor = cm.doc.getCursor();
        
        // read a content from a line where is the current cursor
        
        var line_content = cm.doc.getLine(current_cursor.line);
        cm.execCommand('goLineEnd');
        cm.execCommand('newlineAndIndent');
        cm.execCommand('indentLess');
        cm.doc.replaceSelection(line_content);
        cm.doc.setCursor(current_cursor.line + 1, current_cursor.ch);
        return false;
    }}
);

<IPython.core.display.Javascript object>

### DCG function

Implementation for the discounted cumulative gain (DCG) function.


$$ \mathrm{DCG_{p}} = \sum_{i=1}^{N} \frac{rel_{i}}{\log_{b}(i+1)} = rel_1 + \sum_{i=2}^{N} \frac{rel_{i}}{\log_{b}(i+1)}  $$

Mean average precision (MAP) 

$$ F_{1}=\frac{\text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}} $$  

F1 score (only defined for binary relevance scores).

$$ \operatorname{MAP} = \frac{\sum_{q=1}^N \operatorname{AveP(q)}}{N} $$ 


<br> 




In [3]:
def dcg(y_true: np.array, log_base: 2):
    """
    Returns ndcg given the truth values (clicks) for a given query
    :param y_true: np.array of clicked documents in order
    :param log_base: the log discount factor
    :return:
    """
    gain = y_true[0]
    for index, rel_i in enumerate(y_true):
        if index !=0:
            gain += rel_i / (math.log(rel_i+index+1, log_base))
    return gain


In [4]:
def request_into_dataframe(rows= 100, query= '*:*', sort='',start=0) -> pd.DataFrame :
    """
    :param rows: amount of rows to request
    :param query: string to query
    :param sort: string to sort the request eg. sort='ShortTimeStamp desc'
    """

    url = 'http://ec2-18-184-94-154.eu-central-1.compute.amazonaws.com:8080/solr' \
    '/ACC_Logging_Slave/select?indent=on&q={q}&rows={r}&sort={s}&start={st}&wt=json'.format(q=query,r=rows,s=sort,st=start)

    response = requests.get(url)
    
    # Transform the request into a json
    try:
        response = response.json()
        response = response['response']['docs'][:]
    except:
        display(response)
        
    df =  pd.DataFrame(response)
    
    return df
    


In [5]:
def get_dataframe_232(rows=100000000, from_disk=False, save_name='df_232') -> pd.DataFrame:
    """
    Get all the logging with a clicked documentPosition. 
    DocumentPosition clicked is logged since 01-07-2019 -> 20190701
    :param rows: query 
    :param from_disk: query string
    :param save_name: query string
    :return:
    """
    
    directory = './data/{}'.format(save_name)

    
    if from_disk:
        return pd.read_hdf(directory)
    else:
        df_232 = request_into_dataframe(rows=rows, query='EventID:232 & sort=ShortTimeStamp desc')
        
        # Get query of reponsehader into a seperate column. 
        df_232['SearchText'] = df_232['ResponseHeader'].progress_apply(lambda x: json.loads(x)['params']['userQuery'][:])
    
        #   Transform documentIDs from string to list of strings. 
        df['DocumentIDs'] = df['DocumentIDs'].progress_apply(lambda x: str(x).split())    

        if not os.path.exists('./data'):
            os.makedirs(directory)
    
        df_232.to_hdf(directory, key='test', mode='w')
        
    return df_232 


def get_dataframe_27(rows=1000000000, from_disk=False, save_name='df_27') -> pd.DataFrame:
    """
    Get all the logging with a clicked documentPosition. 
    DocumentPosition clicked is logged since 01-07-2019 -> 20190701
    :param rows: query 
    :param from_disk: query string
    :param save_name: query string
    :return:
    """
      
    directory = './data/{}'.format(save_name)
        
    if from_disk:
        return pd.read_hdf(directory)
    
    else:
        print("Requesting dataframe 27")
        df_27 = request_into_dataframe(rows=rows, query='EventID:27 AND ShortTimeStamp:[20190701 TO 20201201] DocumentPosition:* AND SearchText:*')
        df_27.to_hdf(directory, key='test', mode='w')
        print("Retrieved dataframe 27")

    return df_27 


def check_id_list(clicked_pos: int, document_id: int, retrieved_ids: []):
    """
    Check if the clicked pos matches a the corresponding document ID
    """
    
    retrieved_ids = [int(y) for y in retrieved_ids[0].split()]
    clicked_pos = int(clicked_pos) - 1 
    document_id = int(document_id)

    try:
        return retrieved_ids[clicked_pos] == document_id
    except:
        print("Document ID, Clicked pos, retrieved_id ", document_id, clicked_pos, retrieved_ids)

def check_in_timerange(df_232_group = pd.DataFrame, df_27 = pd.DataFrame) -> list:
    """
    Check if the there are clickedobjects within + and - 5 minutes from the query
    timestamp: object from the dateutil parser library
    :param timestamp:
    :param query:
    :param df_27:
    """
    
    timestamps_27 = [parser.parse(time, fuzzy_with_tokens=False) for time in df_27['TimeStamp'].tolist()]    
    stamp_232 = parser.parse(df_232_group['TimeStamp'].iloc[0], fuzzy_with_tokens=False)
    
    retrieved_ids = df_232_group['DocumentIDs'].tolist()
    
    doc_pos = []
    doc_ids = []
    
    for idx, stamp_27 in enumerate(timestamps_27):        
        if (stamp_27 in DateTimeRange(stamp_232, stamp_232  + timedelta(minutes=20))):
            
            doc_id = df_27['DocumentID'].iloc[idx]
            pos    = df_27['DocumentPosition'].iloc[idx]
           
            if (np.isnan(pos)):
                return
        
            if pos <= 19:
                if check_id_list(clicked_pos=pos, document_id=doc_id, retrieved_ids=retrieved_ids):
                    doc_pos.append(df_27['DocumentPosition'].iloc[idx])
                    doc_ids.append(df_27['DocumentID'].iloc[idx])
            else:
                doc_pos.append(df_27['DocumentPosition'].iloc[idx])
                doc_ids.append(df_27['DocumentID'].iloc[idx])

    return doc_pos, doc_ids


def check_and_concat(df_232: pd.DataFrame, df_27: pd.DataFrame) -> pd.DataFrame:
    """
    Check both dataframes get clicks from 27 and add these to responseheader taken from 232
    :param df_232: dataframe containing the 232 log
    :param df_27: dataframe containing the 27 log
    """
    
    final_pos = [] 
    final_ids = []
        
    for idx, (df_shape, df_232_groupby) in enumerate(tqdm(df_232.groupby(by=['UserID','ShortTimeStamp','ID']) , desc='Concatenating clicks')):
        
        # Only check within the part of dataframe 27 that has similar UserID and SearchText
        temp_27 = df_27[(df_27['UserID'] == df_232_groupby['UserID'].iloc[0]) & (df_27['SearchText'] == df_232_groupby['SearchText'].iloc[0])]
    
        doc_pos = []
        doc_ids = []
        
        if not temp_27.empty:
            doc_pos, doc_ids = check_in_timerange(df_232_groupby, temp_27)
            final_pos.append(doc_pos)
            final_ids.append(doc_ids)
        else:
            # No clicks found
            final_pos.append([])
            final_ids.append([])

            
    # Get the ordering we found from the GroupBy ordering from the for loop above
    df_232 = df_232.sort_values(by=['UserID','ShortTimeStamp','ID'])

    df_232['ClickedPos'] = final_pos
    df_232['ClickedIDs'] = final_ids
        
    return df_232



## Help functions to read in all of EventID:232

Not necessary for workflow but could be used to read in the full EventID:232 dataframe
<br>

In [6]:
def read_in_232():
    """
    Request 200 000 rows from EventID:232 and saves it in ./data/
    """
    for i in range(0,200000,50000):
        print(i)
        df = request_into_dataframe(rows=50000,query='EventID:232',start=i)
        df.to_hdf('./data/EventID:232_start={}'.format(i), key='test', mode='w')

def read_in_dataframe():
    """
    Concats all the frames into big dataframe 200 000 rows from EventID:232 and saves it in ./data/
    """

    all_frames = ['./data/EventID:232_0_to_50000','./data/EventID:232_all_50000_to_80000', 
             './data/EventID:232_all_80000_to_130000','./data/EventID:232_all_130000_to_170000']    

    df = pd.concat(map(pd.read_hdf, all_frames))
    df.to_hdf('./data/concatenated_frames' ,  key='test', mode='w')

In [7]:
def find_responses_232(df_reproduce: pd.DataFrame) -> pd.DataFrame:
    """
    Find the appropriate events from dataframe 232 belonging to the query set in df_reproduce.
    :param df_reproduce: dataframe to reproduce the queries from
    """
    
    response_list = [] 

    for i in tqdm(range(len(df_reproduce)), desc="Finding responses in EventID:232"):

        df_locked = df_reproduce.iloc[i]

        userid = df_locked['UserID'] 
        timestamp = df_locked['TimeStamp'] 
        total_results = df_locked['TotalResults']
        stamp  = timestamp.split('T')
        stamp =  stamp[0].replace('-','')

        df_232 = request_into_dataframe(rows=5000, query='EventID:232 AND ShortTimeStamp:{} AND UserID:{}'.format(stamp,userid))
        df_232 =  df_232[(df_232['TimeStamp'] == timestamp) & (df_232['TotalResult'] == total_results)]
        df_232['SearchText'] = df_232['ResponseHeader'].apply(lambda x: json.loads(x)['params']['userQuery'][:])
        response_list.append(df_232)

    df = pd.concat(response_list)
    return df


In [8]:
def make_params(x: str, disable_boost=True):
    """
    Reformat the parameter string to be curlable
    :param x: parameter string 
    """

#     Disable these boosting factors
    if disable_boost:
        x['legislationBoostQ'] = "ID:0" 
        x['lowRankQ'] = "ID:0" 
        x['newsQ'] = "ID:0" 

    new_query = []
    for key,value in x.items():
        if key == 'fq':
            for fq_value in value:
                fq_value = urllib.parse.quote(string=str(fq_value))
                new_query.append("&{}={}".format('fq',fq_value))
        elif key == 'shards':
            value = value.replace("PRD_","ACC_")
            value = urllib.parse.quote(string=str(value))
            new_query.append("&{}={}".format(key,value))
        elif key == 'json.facet':
            pass
        else:
            value = urllib.parse.quote(string=str(value))
            new_query.append("&{}={}".format(key,value))


    params = ''.join(new_query)
    
    
    return params

In [9]:
import urllib.request
import time 

def internet_on():
    try:
        # Try if google.com is live
        urllib.request.urlopen('http://216.58.192.142', timeout=1)
        return True
    except urllib.request.URLError as err: 
        return False

def loop_through_queries(df: pd.DataFrame):
    """
    Loop through the queries
    :param df: dataframe with a responseheader to replicate the clicks
    """
    


    df['replicated_ids'] = pd.Series()
    df['replicated_scores'] = pd.Series()

    all_doc_ids = []
    all_doc_scores = []
    
    
    for i in tqdm(range(len(df)), desc='Replicating clicks'):
        
        if not internet_on():
            print("Connection failure")
        
        df_locked = df.iloc[i]
        
        os.popen('rm /home/winston/LegalIntelligence/all_curl/params/*')
        os.popen('rm /home/winston/LegalIntelligence/all_curl/responses/*')
        
        try:
            # Transform the responseheader to a json
            x = json.loads(df_locked['ResponseHeader'])
            # Transform the params
            params = make_params(x['params'])


            directory = "./all_curl/params/"
            pathlib.Path(directory  + 'param_{}.txt'.format(i)).write_text(params)
            os.popen('curl --data-binary @./all_curl/params/param_{}.txt http://ec2-18-184-94-154.eu-central-1.compute.amazonaws.com:8080/solr/ACC_Legal_Slave/select \
                                                                          > ./all_curl/responses/response_{}.txt'.format(i,i))

            # Take 5 seconds to wait for the curl to get parameters
            time.sleep(5)
            with open(os.getcwd()  + "/all_curl/responses/response_{}.txt".format(i)) as jsonfile:
                data = json.load(jsonfile)

            docids = []
            scores = []

            for idx,document in enumerate(data['response']['docs']):
                docids.append(document['ID'])
                scores.append(document['score'])
            
            all_doc_ids.append(docids)
            all_doc_scores.append(scores)

            df.iloc[i]['replicated_ids'] = docids
            df.iloc[i]['replicated_scores'] = scores
        except:
            print("exception for doc : ", i)
            docids= ['in exception']
            scores= ['in exception']
            all_doc_ids.append(docids)
            all_doc_scores.append(scores)
            df.iloc[i]['replicated_ids'] = docids
            df.iloc[i]['replicated_scores'] = scores
    
    
    df['replicated_ids'] = all_doc_ids
    df['replicated_scores'] = all_doc_scores
    
    return df
    
 

# Start workflow


* 1. Read in the queries that need to be reproduced from an xlsx excel file. These are being read from ./data/ folder so make sure to have that folder with the correct filename
* 2. Find the correct events from EventID:232 that correspond with these queries, this is done in function: **find_responses_232()**
* 3. Request dataframe 27 to find clicks to match
      This is done in **get_dataframe_27(from_disk=False) -> turn from_disk to False if its the first time to run**. After that a copy is stored in ./data/
      and this parameter can be set True
* 4. Check and concat all the query sessions from EventID:232 to the potential clicks found in dataframe 27
* 5. Finally loop through all the queries to replicate them and append the scores and clicks to the dataframe
* 6. Save the replicated query sessions to a excel file
<br>

Ignore pandas FutureWarinings for now. 



In [13]:
# df_reproduce = pd.read_excel('./data/Queries Rank Monitor Reproduceer file.xlsx', header=0, encoding = 'unicode_escape')
# df_232 = find_responses_232(df_reproduce)
# df_27  = get_dataframe_27(from_disk=True)
# df     = check_and_concat(df_232=df_232, df_27=df_27)
# df     = loop_through_queries(df)
df


,AppID,CompID,Department,DocumentIDs,EventID,FunctionArea,ID,Month,Quarter,ResponseHeader,SearchText,ShortTimeStamp,SiteID,TimeStamp,TotalResult,UserID,Week,Year,_version_,ClickedPos,ClickedIDs
5,1,471,NaN,31999035 31991880 32245563 31516798 31422938 3...,232,NaN,1489e398-a381-4b74-a4b6-23f10d1015e7,7,3,"{""li.ext"":[[""li.queryterms"",[[""word"",[[[""term""...",airbnb,20190712,1,2019-07-12T18:01:51.156Z,797,16217,28,2019,1638876589318995968,[1],[31999035]
0,1,547,OR,31561098 30766500 30882889 28479984 13700828 3...,232,NaN,8c0ed5f1-1ec5-45c4-9c61-62fdcfbdeedc,8,3,"{""li.ext"":[[""li.queryterms"",[[""word"",[[[""term""...",docusign,20190806,1,2019-08-06T09:37:02.454Z,16,22804,32,2019,1641109753497124864,"[1, 3, 5, 11]","[31561098, 30882889, 13700828, 4440362]"
15,1,791,800171 AMS Knowledge & Information,32313948 32317567 32317260 32317279 32317331 3...,232,NaN,530b1eef-88f5-4601-bc23-010ad50b2674,7,3,"{""li.ext"":[[""li.queryterms"",[[""word"",[[[""term""...",fintech,20190725,1,2019-07-25T07:40:12.163Z,1030,36905,30,2019,1640015238995640320,"[2, 3]","[32317567, 32317260]"
23,1,822,Civilisten,32150904 22008615 30075668 32204245 32120935 1...,232,NaN,1197fc0f-8fa6-414f-8f61-04905e7bac24,8,3,"{""li.ext"":[[""li.queryterms"",[[""concept"",[[[""ID...",letter of intent,20190801,1,2019-08-01T11:27:11.338Z,758,40860,31,2019,1640663698518310912,"[3, 24]","[30075668, 12488532]"
46,1,822,Civilisten,32317649 15475531 32150904 23695638 22008615 1...,232,NaN,b953a320-c7b4-4a40-8326-79fb10a77705,8,3,"{""li.ext"":[[""li.queryterms"",[[""concept"",[[[""ID...",letter of intent,20190801,1,2019-08-01T11:16:01.435Z,2846,40860,31,2019,1640662996082491393,"[3, 24, 5]","[32150904, 12488532, 22008615]"
13,1,349,NaN,32249281 32233288 32240581 32235147 32236480 3...,232,NaN,7d6da77a-564c-4f35-9cb3-4e6d27452171,7,3,"{""li.ext"":[[""li.queryterms"",[[""word"",[[[""term""...",wettelijke bedenktijd,20190715,1,2019-07-15T06:48:44.647Z,3684,61617,29,2019,1639106093832470528,"[71, 75, 53, 24, 18, 7, 19, 48, 50, 49, 50, 58...","[32004684, 31873842, 32038254, 32133735, 32111..."
7,1,349,NaN,31744035 32101994 32333312 32145472 32330780 3...,232,NaN,3036c7bc-97ee-46e0-960e-551f025066aa,7,3,"{""li.ext"":[[""li.queryterms"",[[""word"",[[[""term""...",slapend dienstverband,20190731,1,2019-07-31T14:35:52.486Z,3900,65367,31,2019,1640584972782796800,"[50, 2, 32, 3, 53, 52, 7, 3, 30, 80, 209]","[32335361, 32101994, 32325284, 32333312, 32120..."
30,1,109,Litigation,32320669 32320112 32313485 32118754 32299936 3...,232,NaN,1d8a9f36-d010-4bb9-b0cd-01143a7bc8b2,7,3,"{""li.ext"":[[""li.queryterms"",[[""concept"",[[[""ID...",borgtocht,20190729,1,2019-07-29T09:20:47.547Z,10243,79146,31,2019,1640383955639205888,"[2, 10, 287, 25, 23, 237]","[32320112, 32072194, 31278829, 32234998, 32237..."
0,1,109,Litigation,32322897 32218983 32321088 32319318 32319680 3...,232,NaN,1e2699ac-2b39-432c-b78f-74062d528349,7,3,"{""li.ext"":[[""li.queryterms"",[[""concept"",[[[""ID...",bankgarantie,20190729,1,2019-07-29T10:03:17.885Z,22689,79146,31,2019,1640386629759664128,[1],[32322897]
4,1,1239,Ressortsparket,32236510 32142467 32135060 32222411 32227499 3...,232,NaN,599c173d-b5e3-4895-ba9f-5800e9a406f3,7,3,"{""li.ext"":[[""li.queryterms"",[[""concept"",[[[""ID...",voorwaardelijke invrijheidstelling,20190719,1,2019-07-19T09:43:32.395Z,5671,85958,29,2019,1639479416779177984,[41],[32236510]


In [18]:
dummy = df_27[(df_27['UserID'] == 98472) & (df_27['ShortTimeStamp'] == 20190805)]
dummy.sort_values(by=['TimeStamp'])

,ID,TimeStamp,ShortTimeStamp,Year,Week,Quarter,Month,EventID,SearchTypeID,UserID,DocumentURL,DocumentSource,DocumentID,AppID,Publisher,CompID,SiteID,SearchText,DocumentPosition,RunID,_version_,Department,FunctionArea
329543,43063c06-3f79-4bac-ad50-f5ef254a5a8a,2019-08-05T07:41:23.204Z,20190805,2019,32,3,8,27,14,98472,https://www.navigator.nl/#/document/id1cf38d7b...,AB,32321075,1,Kluwer,39,1,A.C. Rop,1,81742370-0d45-4997-b85d-8560b0ea8c2d,1641011900771729408,Dir. JZ,NaN
332383,9d616e72-1c5d-4b27-986e-367ca60999c0,2019-08-05T07:49:02.055Z,20190805,2019,32,3,8,27,14,98472,https://www.navigator.nl/#/document/id2ec9df25...,Prg.,31402848,1,Kluwer,39,1,A.C. Rop,32,3f02264f-75b0-4e5b-90ad-4d919b338927,1641012361284288512,Dir. JZ,NaN
329154,776581ca-b922-42e3-ab5f-79e56c00ca0e,2019-08-05T08:22:01.939Z,20190805,2019,32,3,8,27,14,98472,http://deeplink.rechtspraak.nl/uitspraak?id=EC...,Rechtspraak.nl,31186273,1,Diversen,39,1,A.C. Rop,33,3f02264f-75b0-4e5b-90ad-4d919b338927,1641014437270781952,Dir. JZ,NaN
329471,e4ca43fd-a4b6-4b48-b972-a44ed1c4e813,2019-08-05T08:23:48.861Z,20190805,2019,32,3,8,27,14,98472,http://deeplink.rechtspraak.nl/uitspraak?id=EC...,Rechtspraak.nl,30780305,1,Diversen,39,1,A.C. Rop,34,3f02264f-75b0-4e5b-90ad-4d919b338927,1641014549383479296,Dir. JZ,NaN
329487,713fd439-8ab0-4ece-9e36-5b49d96af171,2019-08-05T08:24:26.047Z,20190805,2019,32,3,8,27,14,98472,http://deeplink.rechtspraak.nl/uitspraak?id=EC...,Rechtspraak.nl,31165978,1,Diversen,39,1,A.C. Rop,35,3f02264f-75b0-4e5b-90ad-4d919b338927,1641014588375826432,Dir. JZ,NaN
369150,554dbab4-39bd-4917-8d41-77990d43bf3f,2019-08-05T08:25:15.857Z,20190805,2019,32,3,8,27,14,98472,http://deeplink.rechtspraak.nl/uitspraak?id=EC...,Rechtspraak.nl,31065209,1,Diversen,39,1,A.C. Rop,36,3f02264f-75b0-4e5b-90ad-4d919b338927,1641014640605396992,Dir. JZ,NaN
369178,30fe90e1-3232-4b61-bb00-f317ebae960e,2019-08-05T08:26:39.538Z,20190805,2019,32,3,8,27,14,98472,http://deeplink.rechtspraak.nl/uitspraak?id=EC...,Rechtspraak.nl,30976538,1,Diversen,39,1,A.C. Rop,37,3f02264f-75b0-4e5b-90ad-4d919b338927,1641014728352333824,Dir. JZ,NaN
369499,a0bdc8eb-de3e-4c8c-a1df-79350a5c9df5,2019-08-05T08:32:34.406Z,20190805,2019,32,3,8,27,14,98472,http://deeplink.rechtspraak.nl/uitspraak?id=EC...,Rechtspraak.nl,30968472,1,Diversen,39,1,A.C. Rop,38,3f02264f-75b0-4e5b-90ad-4d919b338927,1641015100452110336,Dir. JZ,NaN
369514,9b28b13c-8fd2-428c-95e2-95528e038d48,2019-08-05T08:33:28.877Z,20190805,2019,32,3,8,27,14,98472,http://deeplink.rechtspraak.nl/uitspraak?id=EC...,Rechtspraak.nl,30886951,1,Diversen,39,1,A.C. Rop,41,5c28bb93-b00e-4fa6-a254-32a7a6856e28,1641015157565947904,Dir. JZ,NaN
369287,c723bbda-fbed-4344-ab7e-f84a545bf17c,2019-08-05T08:34:01.232Z,20190805,2019,32,3,8,27,14,98472,http://deeplink.rechtspraak.nl/uitspraak?id=EC...,Rechtspraak.nl,30883420,1,Diversen,39,1,A.C. Rop,42,5c28bb93-b00e-4fa6-a254-32a7a6856e28,1641015191492624384,Dir. JZ,NaN


## Save the dataframe as an excle file 

<br>

In [21]:
df.iloc[16]['DocumentIDs']
# df.to_excel('./data/appended_with_clicks')

'31371993 31455087 31226734 31172531 31071268 31035648 31033655 31084583 31003580 30955985 30954253 30944000 30894198 30886326 30873698 30886481 30829751 31417578 30765872 30783965'